This notebook shows sprinkling result in SR1 data, with single electrons sprinkled with realistic timing and XY from data. Since we loads lots of data with even waveforms, you'd better get 40GB for this notebook.

It is expected to be running in `2024.03.1` container. 

There will be 3 datasets in the end:
- `data`: Exactly the same as `v14` offline real data.
- `simulation`: Events reconstructed using the simulation instruction only, there is nothing else in the reconstruction process.
- `sprinkled`: Events reconstructed by mixing simulation and data. Some time it is also called `salt` and they mean the same thing.

The source of simulation instruction from data SEs selected with a random 0.1FDT offset is here at `/project/lgrandi/yuanlq/salt/se_instructions/`. Unlike AmBe case, here the sprinkled SEs are already having fixed instruction in directories above, with a rate of 200Hz.

Lanqing, Mar 26 2024 (Editted Apr 04 2024)

# Preparation

In [ ]:
import saltax
import straxen
import cutax
from tqdm import tqdm
import gc
import numpy as np
import utilix
import matplotlib.pyplot as plt
from tabulate import tabulate
from glob import glob
from itertools import cycle
from saltax.match.utils import *

straxen.print_versions()

In [ ]:
# Define contexts for sprinkling mode and simulation mode respectively
st_salt = saltax.contexts.sxenonnt(
    saltax_mode="salt",
    output_folder="/project/lgrandi/yuanlq/salt/se_bootstrapped",
    faxconf_version="sr0_v4",
    generator_name="se_bootstrapped",
    recoil=8,
    mode="all",
)
st_simu = saltax.contexts.sxenonnt(
    saltax_mode="simu",
    output_folder="/project/lgrandi/yuanlq/salt/se_bootstrapped",
    faxconf_version="sr0_v4",
    generator_name="se_bootstrapped",
    recoil=8,
    mode="all",
)

# This is the regular straxen context to load data
st_data = cutax.xenonnt_offline()

In [ ]:
runs_with_rawdata = saltax.find_runs_with_rawdata(
    rawdata_folders=[
        "/project/lgrandi/yuanlq/salt/raw_records/",
        "/scratch/midway2/yuanlq/salt/raw_records/",
        "/scratch/midway3/yuanlq/salt/raw_records/",
    ]
)
runs_with_rawdata

In [ ]:
saltax.get_available_runs(
    runs_with_rawdata,
    st_salt,
    st_simu,
    salt_available=["peak_basics", "peak_positions_mlp"],
    simu_available=[],
)

In [ ]:
kr83m = ["053167"]
rn220 = ["048693", "048692", "048698", "049432", "049433"]
ybe = ["047876"]

In [ ]:
peaks = st_simu.get_array("053167", "peaks", seconds_range=(0, 2))
plt.figure(dpi=150)
for p in peaks:
    plt.plot(np.arange(200) * p["dt"], p["data"])
plt.xlabel("Time [ns]")
plt.ylabel("Amplitude [PE/10ns]")
plt.xlim(0, 2000)
plt.title("Example Single Electrons Simulated")

Good, on the [special wfsim version](https://github.com/XENONnT/WFSim/pull/434) our SEs look good. If it looks like combs at 250ns resolution, it means you are using the wrong `wfsim` or `fuse` version.

# Let's sprinkle

## Rn220

In [ ]:
(peaks_simu, peaks_salt, inds_dict) = load_peaks(rn220, st_salt, st_simu);

In [ ]:
# Now peaks_salt_matched_to_simu and peaks_simu_matched_to_salt are 1-1 corresponding
peaks_salt_matched_to_simu = peaks_salt[inds_dict["ind_salt_peak_found"]]
peaks_simu_matched_to_salt = peaks_simu[inds_dict["ind_simu_peak_found"]]

# Further filter out the ones whose simu fail daq cut
mask_simu_daq_cut = saltax.apply_peaks_daq_cuts(st_data, rn220, peaks_simu_matched_to_salt)
peaks_salt_matched_to_simu = peaks_salt_matched_to_simu[mask_simu_daq_cut]
peaks_simu_matched_to_salt = peaks_simu_matched_to_salt[mask_simu_daq_cut]

In [ ]:
plt.figure(dpi=150)
plt.hist(
    peaks_salt_matched_to_simu["area"],
    bins=np.linspace(0, 100, 101),
    histtype="step",
    color="tab:blue",
    label="Matched Sprinkled: %sPE"
    % (np.round(np.median(peaks_salt_matched_to_simu["area"]), decimals=2)),
)
plt.hist(
    peaks_simu_matched_to_salt["area"],
    bins=np.linspace(0, 100, 101),
    histtype="step",
    color="tab:red",
    label="Matched Simulated: %sPE"
    % (np.round(np.median(peaks_simu_matched_to_salt["area"]), decimals=2)),
)
plt.title("Before Cuts SE Ambience Interference in SR1 Rn220")
plt.legend()
plt.xlabel("Area [PE]")
plt.ylabel("Counts [AU]")

In [ ]:
plt.figure(dpi=150)
plt.hist(
    peaks_salt_matched_to_simu["area"] - peaks_simu_matched_to_salt["area"],
    bins=np.linspace(-1, 60, 201),
    histtype="step",
    color="tab:blue",
)
# plt.legend()
plt.xlabel("Area Sprinkled-Simulated [PE]")
plt.ylabel("Counts [AU]")
plt.title("Before Cuts SE Ambience Interference in SR1 Rn220")
plt.yscale("log")

In [ ]:
plt.figure(dpi=150)
plt.hist2d(
    peaks_simu_matched_to_salt["x_mlp"],
    peaks_simu_matched_to_salt["y_mlp"],
    bins=(np.linspace(-65, 65, 100), np.linspace(-65, 65, 100)),
)
plt.xlabel("X [cm]")
plt.ylabel("Y [cm]")
plt.title("Before Cuts Selected SE in SR1 Rn220")
plt.show()

In [ ]:
plt.figure(dpi=150)
plt.hist(
    peaks_salt_matched_to_simu["range_50p_area"],
    bins=np.linspace(0, 5e3, 100),
    histtype="step",
    color="tab:blue",
    label="Matched Sprinkled",
)
plt.hist(
    peaks_simu_matched_to_salt["range_50p_area"],
    bins=np.linspace(0, 5e3, 100),
    histtype="step",
    color="tab:red",
    label="Matched Simulated",
)
plt.yscale("log")
plt.legend()
plt.xlabel("50p width [ns]")
plt.title("Before Cuts SE Ambience Interference in SR1 Rn220")

In [ ]:
plt.figure(dpi=150)
after_cuts_salt = peaks_salt_matched_to_simu[peaks_salt_matched_to_simu["range_90p_area"] < 2000]
after_cuts_simu = peaks_simu_matched_to_salt[peaks_salt_matched_to_simu["range_90p_area"] < 2000]
plt.hist(
    after_cuts_salt["area"],
    bins=np.linspace(0, 100, 100),
    histtype="step",
    color="tab:blue",
    label="Matched Sprinkled: %sPE"
    % (
        np.round(np.median(after_cuts_salt["area"][~np.isnan(after_cuts_salt["area"])]), decimals=2)
    ),
)
plt.hist(
    after_cuts_simu["area"],
    bins=np.linspace(0, 100, 100),
    histtype="step",
    color="tab:red",
    label="Matched Simulated: %sPE"
    % (
        np.round(np.median(after_cuts_simu["area"][~np.isnan(after_cuts_simu["area"])]), decimals=2)
    ),
)
plt.title("90p Width < 2000ns SE Ambience Interference in SR1 Rn220")
plt.legend()
plt.xlabel("Area [PE]")
plt.ylabel("Counts [AU]")

## YBe

In [ ]:
(peaks_simu, peaks_salt, inds_dict) = load_peaks(ybe, st_salt, st_simu);

In [ ]:
# Now peaks_salt_matched_to_simu and peaks_simu_matched_to_salt are 1-1 corresponding
peaks_salt_matched_to_simu = peaks_salt[inds_dict["ind_salt_peak_found"]]
peaks_simu_matched_to_salt = peaks_simu[inds_dict["ind_simu_peak_found"]]

# Further filter out the ones whose simu fail daq cut
mask_simu_daq_cut = saltax.apply_peaks_daq_cuts(st_data, ybe, peaks_simu_matched_to_salt)
peaks_salt_matched_to_simu = peaks_salt_matched_to_simu[mask_simu_daq_cut]
peaks_simu_matched_to_salt = peaks_simu_matched_to_salt[mask_simu_daq_cut]

In [ ]:
plt.figure(dpi=150)
plt.hist(
    peaks_salt_matched_to_simu["area"],
    bins=np.linspace(0, 100, 101),
    histtype="step",
    color="tab:blue",
    label="Matched Sprinkled: %sPE"
    % (np.round(np.median(peaks_salt_matched_to_simu["area"]), decimals=2)),
)
plt.hist(
    peaks_simu_matched_to_salt["area"],
    bins=np.linspace(0, 100, 101),
    histtype="step",
    color="tab:red",
    label="Matched Simulated: %sPE"
    % (np.round(np.median(peaks_simu_matched_to_salt["area"]), decimals=2)),
)
plt.title("Before Cuts SE Ambience Interference in SR1 YBe")
plt.legend()
plt.xlabel("Area [PE]")
plt.ylabel("Counts [AU]")

In [ ]:
plt.figure(dpi=150)
plt.hist(
    peaks_salt_matched_to_simu["area"] - peaks_simu_matched_to_salt["area"],
    bins=np.linspace(-1, 60, 201),
    histtype="step",
    color="tab:blue",
)
# plt.legend()
plt.xlabel("Area Sprinkled-Simulated [PE]")
plt.ylabel("Counts [AU]")
plt.title("Before Cuts SE Ambience Interference in SR1 YBe")
plt.yscale("log")

In [ ]:
plt.figure(dpi=150)
plt.hist2d(
    peaks_simu_matched_to_salt["x_mlp"],
    peaks_simu_matched_to_salt["y_mlp"],
    bins=(np.linspace(-65, 65, 100), np.linspace(-65, 65, 100)),
)
plt.xlabel("X [cm]")
plt.ylabel("Y [cm]")
plt.title("Before Cuts Selected SE in SR1 YBe")
plt.show()

In [ ]:
plt.figure(dpi=150)
plt.hist(
    peaks_salt_matched_to_simu["range_50p_area"],
    bins=np.linspace(0, 5e3, 100),
    histtype="step",
    color="tab:blue",
    label="Matched Sprinkled",
)
plt.hist(
    peaks_simu_matched_to_salt["range_50p_area"],
    bins=np.linspace(0, 5e3, 100),
    histtype="step",
    color="tab:red",
    label="Matched Simulated",
)
plt.yscale("log")
plt.legend()
plt.xlabel("50p width [ns]")
plt.title("Before Cuts SE Ambience Interference in SR1 YBe")

In [ ]:
plt.figure(dpi=150)
after_cuts_salt = peaks_salt_matched_to_simu[peaks_salt_matched_to_simu["range_90p_area"] < 2000]
after_cuts_simu = peaks_simu_matched_to_salt[peaks_salt_matched_to_simu["range_90p_area"] < 2000]
plt.hist(
    after_cuts_salt["area"],
    bins=np.linspace(0, 100, 100),
    histtype="step",
    color="tab:blue",
    label="Matched Sprinkled: %sPE"
    % (
        np.round(np.median(after_cuts_salt["area"][~np.isnan(after_cuts_salt["area"])]), decimals=2)
    ),
)
plt.hist(
    after_cuts_simu["area"],
    bins=np.linspace(0, 100, 100),
    histtype="step",
    color="tab:red",
    label="Matched Simulated: %sPE"
    % (
        np.round(np.median(after_cuts_simu["area"][~np.isnan(after_cuts_simu["area"])]), decimals=2)
    ),
)
plt.title("90p Width < 2000ns SE Ambience Interference in SR1 YBe")
plt.legend()
plt.xlabel("Area [PE]")
plt.ylabel("Counts [AU]")

## Kr83m

In [ ]:
(peaks_simu, peaks_salt, inds_dict) = load_peaks(kr83m, st_salt, st_simu);

In [ ]:
# Now peaks_salt_matched_to_simu and peaks_simu_matched_to_salt are 1-1 corresponding
peaks_salt_matched_to_simu = peaks_salt[inds_dict["ind_salt_peak_found"]]
peaks_simu_matched_to_salt = peaks_simu[inds_dict["ind_simu_peak_found"]]

# Further filter out the ones whose simu fail daq cut
mask_simu_daq_cut = saltax.apply_peaks_daq_cuts(st_data, kr83m, peaks_simu_matched_to_salt)
peaks_salt_matched_to_simu = peaks_salt_matched_to_simu[mask_simu_daq_cut]
peaks_simu_matched_to_salt = peaks_simu_matched_to_salt[mask_simu_daq_cut]

In [ ]:
plt.figure(dpi=150)
plt.hist(
    peaks_salt_matched_to_simu["area"],
    bins=np.linspace(0, 100, 101),
    histtype="step",
    color="tab:blue",
    label="Matched Sprinkled: %sPE"
    % (np.round(np.median(peaks_salt_matched_to_simu["area"]), decimals=2)),
)
plt.hist(
    peaks_simu_matched_to_salt["area"],
    bins=np.linspace(0, 100, 101),
    histtype="step",
    color="tab:red",
    label="Matched Simulated: %sPE"
    % (np.round(np.median(peaks_simu_matched_to_salt["area"]), decimals=2)),
)
plt.title("Before Cuts SE Ambience Interference in SR1 Kr83m")
plt.legend()
plt.xlabel("Area [PE]")
plt.ylabel("Counts [AU]")

In [ ]:
plt.figure(dpi=150)
plt.hist(
    peaks_salt_matched_to_simu["area"] - peaks_simu_matched_to_salt["area"],
    bins=np.linspace(-1, 60, 201),
    histtype="step",
    color="tab:blue",
)
# plt.legend()
plt.xlabel("Area Sprinkled-Simulated [PE]")
plt.ylabel("Counts [AU]")
plt.title("Before Cuts SE Ambience Interference in SR1 Kr83m")
plt.yscale("log")

In [ ]:
plt.figure(dpi=150)
plt.hist2d(
    peaks_simu_matched_to_salt["x_mlp"],
    peaks_simu_matched_to_salt["y_mlp"],
    bins=(np.linspace(-65, 65, 100), np.linspace(-65, 65, 100)),
)
plt.xlabel("X [cm]")
plt.ylabel("Y [cm]")
plt.title("Before Cuts Selected SE in SR1 Kr83m")
plt.show()

In [ ]:
plt.figure(dpi=150)
plt.hist(
    peaks_salt_matched_to_simu["range_50p_area"],
    bins=np.linspace(0, 5e3, 100),
    histtype="step",
    color="tab:blue",
    label="Matched Sprinkled",
)
plt.hist(
    peaks_simu_matched_to_salt["range_50p_area"],
    bins=np.linspace(0, 5e3, 100),
    histtype="step",
    color="tab:red",
    label="Matched Simulated",
)
plt.yscale("log")
plt.legend()
plt.xlabel("50p width [ns]")
plt.title("Before Cuts SE Ambience Interference in SR1 Kr83m")

Let's apply a brutal width cut

In [ ]:
plt.figure(dpi=150)
after_cuts_salt = peaks_salt_matched_to_simu[peaks_salt_matched_to_simu["range_90p_area"] < 2000]
after_cuts_simu = peaks_simu_matched_to_salt[peaks_salt_matched_to_simu["range_90p_area"] < 2000]
plt.hist(
    after_cuts_salt["area"],
    bins=np.linspace(0, 100, 100),
    histtype="step",
    color="tab:blue",
    label="Matched Sprinkled: %sPE"
    % (
        np.round(np.median(after_cuts_salt["area"][~np.isnan(after_cuts_salt["area"])]), decimals=2)
    ),
)
plt.hist(
    after_cuts_simu["area"],
    bins=np.linspace(0, 100, 100),
    histtype="step",
    color="tab:red",
    label="Matched Simulated: %sPE"
    % (
        np.round(np.median(after_cuts_simu["area"][~np.isnan(after_cuts_simu["area"])]), decimals=2)
    ),
)
plt.title("90p Width < 2000ns SE Ambience Interference in SR1 Kr83m")
plt.legend()
plt.xlabel("Area [PE]")
plt.ylabel("Counts [AU]")

# Waveforms

Let's watch some waveforms for those who get increased in area for sprinkled dataset. You will need 40GB RAM to run this section.

In [ ]:
# This one loads waveforms so it should be heavy and slow!
(peaks_simu, peaks_salt, inds_dict) = load_peaks(
    ["053167"],
    st_salt,
    st_simu,
    plugins=(
        "peak_basics",
        "peak_positions_mlp",
        "peaks",
    ),  # Just adding peaks so that you have waveforms
);

In [ ]:
# Now peaks_salt_matched_to_simu and peaks_simu_matched_to_salt are 1-1 corresponding
peaks_salt_matched_to_simu = peaks_salt[inds_dict["ind_salt_peak_found"]]
peaks_simu_matched_to_salt = peaks_simu[inds_dict["ind_simu_peak_found"]]

# Further filter out the ones whose simu fail daq cut
mask_simu_daq_cut = saltax.apply_peaks_daq_cuts(st_data, ["053167"], peaks_simu_matched_to_salt)
peaks_salt_matched_to_simu = peaks_salt_matched_to_simu[mask_simu_daq_cut]
peaks_simu_matched_to_salt = peaks_simu_matched_to_salt[mask_simu_daq_cut]

In [ ]:
abnormal_mask = peaks_salt_matched_to_simu["area"] - peaks_simu_matched_to_salt["area"] > 1
ind = 0
plt.figure(dpi=150)
p_salt = peaks_salt_matched_to_simu[abnormal_mask][ind]
p_simu = peaks_simu_matched_to_salt[abnormal_mask][ind]
plt.plot(
    np.arange(200) * p_salt["dt"],
    p_salt["data"] / p_salt["dt"],
    color="tab:blue",
    alpha=0.5,
    label="Sprinkled:%sPE" % (np.round(p_salt["area"], decimals=2)),
)
plt.plot(
    np.arange(200) * p_simu["dt"],
    p_simu["data"] / p_simu["dt"],
    color="tab:red",
    alpha=0.5,
    label="Simulated:%sPE" % (np.round(p_simu["area"], decimals=2)),
)
plt.xlabel("Time [ns]")
plt.ylabel("Amplitude [PE/ns]")
plt.legend()
plt.title("Sprinkled-Simulated Area > 1 PE Waveforms in SR1 Kr83m")

In [ ]:
abnormal_mask = peaks_salt_matched_to_simu["area"] - peaks_simu_matched_to_salt["area"] > 1
ind = 16
plt.figure(dpi=150)
p_salt = peaks_salt_matched_to_simu[abnormal_mask][ind]
p_simu = peaks_simu_matched_to_salt[abnormal_mask][ind]
plt.plot(
    np.arange(200) * p_salt["dt"],
    p_salt["data"] / p_salt["dt"],
    color="tab:blue",
    alpha=0.5,
    label="Sprinkled:%sPE" % (np.round(p_salt["area"], decimals=2)),
)
plt.plot(
    np.arange(200) * p_simu["dt"],
    p_simu["data"] / p_simu["dt"],
    color="tab:red",
    alpha=0.5,
    label="Simulated:%sPE" % (np.round(p_simu["area"], decimals=2)),
)
plt.xlabel("Time [ns]")
plt.ylabel("Amplitude [PE/ns]")
plt.legend()
plt.title("Sprinkled-Simulated Area > 1 PE Waveforms in SR1 Kr83m")

In [ ]:
abnormal_mask = peaks_salt_matched_to_simu["area"] - peaks_simu_matched_to_salt["area"] > 1
ind = 66
plt.figure(dpi=150)
p_salt = peaks_salt_matched_to_simu[abnormal_mask][ind]
p_simu = peaks_simu_matched_to_salt[abnormal_mask][ind]
plt.plot(
    np.arange(200) * p_salt["dt"],
    p_salt["data"] / p_salt["dt"],
    color="tab:blue",
    alpha=0.5,
    label="Sprinkled:%sPE" % (np.round(p_salt["area"], decimals=2)),
)
plt.plot(
    np.arange(200) * p_simu["dt"],
    p_simu["data"] / p_simu["dt"],
    color="tab:red",
    alpha=0.5,
    label="Simulated:%sPE" % (np.round(p_simu["area"], decimals=2)),
)
plt.xlabel("Time [ns]")
plt.ylabel("Amplitude [PE/ns]")
plt.legend()
plt.title("Sprinkled-Simulated Area > 1 PE Waveforms in SR1 Kr83m")